In [1]:
"""
Task 5 - Model Training and Tracking
Complete ML pipeline with MLflow tracking
"""

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
import sys

# Import MLflow and sklearn components
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, 
    f1_score, roc_auc_score, confusion_matrix, 
    classification_report, roc_curve, auc
)

# Import models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

# Import for saving
import joblib
import json
from datetime import datetime

print("=" * 80)
print("TASK 5 - MODEL TRAINING AND TRACKING")
print("Credit Risk Model with MLflow Experimentation")
print("=" * 80)

# -----------------------------
# 1. Setup and Configuration
# -----------------------------
print("\n1. SETTING UP MLFLOW AND CONFIGURATION")
print("-" * 40)

# Set MLflow tracking URI (local directory)
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("credit-risk-modeling")

print("MLflow Configuration:")
print(f"  Tracking URI: {mlflow.get_tracking_uri()}")
print(f"  Experiment: credit-risk-modeling")

# Create directories
os.makedirs('models/task5', exist_ok=True)
os.makedirs('reports', exist_ok=True)
os.makedirs('data/processed/task5', exist_ok=True)

# -----------------------------
# 2. Load and Prepare Data
# -----------------------------
print("\n2. LOADING AND PREPARING DATA")
print("-" * 40)

# Load the data from Task 4
data_path = "data/processed/customers_with_proxy_target_fixed.csv"
if not os.path.exists(data_path):
    print(f"❌ Error: Data file not found at {data_path}")
    print("Please run Task 4 first.")
    sys.exit(1)

print(f"Loading data from {data_path}...")
df = pd.read_csv(data_path)

print(f"✓ Loaded {len(df)} customer records")
print(f"Features: {df.shape[1] - 2} features + CustomerId + Target")

# Separate features and target
X = df.drop(['CustomerId', 'is_high_risk'], axis=1)
y = df['is_high_risk']

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target distribution:")
print(y.value_counts(normalize=True).apply(lambda x: f"{x:.1%}"))

# Check for any missing values
missing = X.isnull().sum().sum()
if missing > 0:
    print(f"⚠ Found {missing} missing values. Imputing with median...")
    X = X.fillna(X.median())
else:
    print("✓ No missing values found")

# -----------------------------
# 3. Train-Test Split
# -----------------------------
print("\n3. CREATING TRAIN-TEST SPLIT")
print("-" * 40)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(df)*100:.1f}%)")
print(f"Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(df)*100:.1f}%)")

print(f"\nClass distribution in training set:")
print(y_train.value_counts(normalize=True).apply(lambda x: f"{x:.1%}"))

print(f"\nClass distribution in test set:")
print(y_test.value_counts(normalize=True).apply(lambda x: f"{x:.1%}"))

# Save the splits
X_train.to_csv('data/processed/task5/X_train.csv', index=False)
X_test.to_csv('data/processed/task5/X_test.csv', index=False)
pd.Series(y_train).to_csv('data/processed/task5/y_train.csv', index=False, header=['is_high_risk'])
pd.Series(y_test).to_csv('data/processed/task5/y_test.csv', index=False, header=['is_high_risk'])

print("✓ Saved train/test splits to data/processed/task5/")

# -----------------------------
# 4. Define Models and Hyperparameters
# -----------------------------
print("\n4. DEFINING MODELS AND HYPERPARAMETERS")
print("-" * 40)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler
joblib.dump(scaler, 'models/task5/scaler.pkl')
print("✓ Saved scaler to models/task5/scaler.pkl")

# Define models with hyperparameter grids
models = {
    "LogisticRegression": {
        "model": LogisticRegression(random_state=42, class_weight='balanced'),
        "params": {
            'C': [0.01, 0.1, 1, 10, 100],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear', 'saga'],
            'max_iter': [1000]
        }
    },
    "RandomForest": {
        "model": RandomForestClassifier(random_state=42, class_weight='balanced'),
        "params": {
            'n_estimators': [100, 200],
            'max_depth': [10, 20, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },
    "GradientBoosting": {
        "model": GradientBoostingClassifier(random_state=42),
        "params": {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'subsample': [0.8, 1.0]
        }
    },
    "DecisionTree": {
        "model": DecisionTreeClassifier(random_state=42, class_weight='balanced'),
        "params": {
            'max_depth': [5, 10, 20, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    }
}

print(f"Defined {len(models)} models for training:")
for model_name in models.keys():
    print(f"  • {model_name}")

# -----------------------------
# 5. Training with MLflow Tracking
# -----------------------------
print("\n" + "=" * 80)
print("5. TRAINING MODELS WITH MLFLOW TRACKING")
print("=" * 80)

best_models = {}
all_results = []

for model_name, model_config in models.items():
    print(f"\n{'='*60}")
    print(f"TRAINING: {model_name}")
    print(f"{'='*60}")
    
    with mlflow.start_run(run_name=model_name):
        # Log parameters
        mlflow.log_param("model", model_name)
        mlflow.log_param("random_state", 42)
        mlflow.log_param("train_size", len(X_train))
        mlflow.log_param("test_size", len(X_test))
        
        # Perform grid search
        print(f"  Performing GridSearchCV...")
        grid_search = GridSearchCV(
            estimator=model_config["model"],
            param_grid=model_config["params"],
            cv=5,
            scoring='roc_auc',
            n_jobs=-1,
            verbose=1
        )
        
        grid_search.fit(X_train_scaled, y_train)
        
        # Get best model
        best_model = grid_search.best_estimator_
        best_models[model_name] = best_model
        
        # Make predictions
        y_pred = best_model.predict(X_test_scaled)
        y_pred_proba = best_model.predict_proba(X_test_scaled)[:, 1]
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        
        # Log metrics
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_metric("roc_auc", roc_auc)
        mlflow.log_metric("best_cv_score", grid_search.best_score_)
        
        # Log parameters
        for param, value in grid_search.best_params_.items():
            mlflow.log_param(f"best_{param}", value)
        
        # Save results
        model_results = {
            "model_name": model_name,
            "accuracy": float(accuracy),
            "precision": float(precision),
            "recall": float(recall),
            "f1_score": float(f1),
            "roc_auc": float(roc_auc),
            "best_cv_score": float(grid_search.best_score_),
            "best_params": grid_search.best_params_,
            "feature_importance": None
        }
        
        # Log feature importance if available
        if hasattr(best_model, 'feature_importances_'):
            feature_importance = dict(zip(X.columns, best_model.feature_importances_))
            model_results["feature_importance"] = feature_importance
            
            # Create feature importance plot
            import matplotlib.pyplot as plt
            plt.figure(figsize=(10, 6))
            sorted_idx = np.argsort(best_model.feature_importances_)[::-1]
            plt.bar(range(len(sorted_idx)), best_model.feature_importances_[sorted_idx])
            plt.xticks(range(len(sorted_idx)), X.columns[sorted_idx], rotation=45, ha='right')
            plt.title(f'{model_name} - Feature Importance')
            plt.tight_layout()
            importance_path = f'reports/feature_importance_{model_name}.png'
            plt.savefig(importance_path)
            plt.close()
            mlflow.log_artifact(importance_path)
        
        # Log model
        mlflow.sklearn.log_model(best_model, f"model_{model_name}")
        
        # Save model locally
        model_path = f'models/task5/{model_name}_best.pkl'
        joblib.dump(best_model, model_path)
        mlflow.log_artifact(model_path)
        
        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        cm_df = pd.DataFrame(cm, 
                           index=['Actual Low Risk', 'Actual High Risk'],
                           columns=['Predicted Low Risk', 'Predicted High Risk'])
        
        cm_path = f'reports/confusion_matrix_{model_name}.csv'
        cm_df.to_csv(cm_path)
        mlflow.log_artifact(cm_path)
        
        all_results.append(model_results)
        
        print(f"\n  ✅ Training complete for {model_name}")
        print(f"     Best CV Score: {grid_search.best_score_:.3f}")
        print(f"     Test Accuracy: {accuracy:.3f}")
        print(f"     Test ROC-AUC: {roc_auc:.3f}")
        print(f"     Best parameters: {grid_search.best_params_}")

# -----------------------------
# 6. Model Comparison and Selection
# -----------------------------
print("\n" + "=" * 80)
print("6. MODEL COMPARISON AND SELECTION")
print("=" * 80)

# Convert results to DataFrame
results_df = pd.DataFrame(all_results)

# Sort by ROC-AUC (primary metric for imbalanced data)
results_df = results_df.sort_values('roc_auc', ascending=False)

print("\nModel Performance Comparison:")
print(results_df[['model_name', 'accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']].to_string(index=False))

# Identify best model
best_model_name = results_df.iloc[0]['model_name']
best_model = best_models[best_model_name]
best_model_metrics = results_df.iloc[0].to_dict()

print(f"\n🎯 BEST MODEL: {best_model_name}")
print(f"   ROC-AUC: {best_model_metrics['roc_auc']:.3f}")
print(f"   Accuracy: {best_model_metrics['accuracy']:.3f}")
print(f"   F1-Score: {best_model_metrics['f1_score']:.3f}")

# Register best model in MLflow
with mlflow.start_run(run_name="Best_Model_Registration"):
    mlflow.log_params(best_model_metrics['best_params'])
    for metric, value in best_model_metrics.items():
        if metric not in ['model_name', 'best_params', 'feature_importance']:
            mlflow.log_metric(metric, value)
    
    # Log the best model
    mlflow.sklearn.log_model(best_model, "best_model")
    
    # Register the model
    model_uri = f"runs:/{mlflow.active_run().info.run_id}/best_model"
    mlflow.register_model(model_uri, "CreditRiskModel")
    
    print(f"✓ Registered {best_model_name} as 'CreditRiskModel' in MLflow Model Registry")

# -----------------------------
# 7. Detailed Evaluation of Best Model
# -----------------------------
print("\n" + "=" * 80)
print(f"7. DETAILED EVALUATION OF BEST MODEL: {best_model_name}")
print("=" * 80)

# Make predictions with best model
y_pred_best = best_model.predict(X_test_scaled)
y_pred_proba_best = best_model.predict_proba(X_test_scaled)[:, 1]

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best, 
                          target_names=['Low Risk', 'High Risk']))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_best)
print(f"\nConfusion Matrix:")
print(f"                Predicted")
print(f"                Low  High")
print(f"Actual Low   {cm[0,0]:5d} {cm[0,1]:5d}")
print(f"Actual High  {cm[1,0]:5d} {cm[1,1]:5d}")

# ROC Curve
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_best)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, 
         label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'ROC Curve - {best_model_name}')
plt.legend(loc="lower right")
roc_path = 'reports/roc_curve_best_model.png'
plt.savefig(roc_path, dpi=150, bbox_inches='tight')
plt.close()
print(f"✓ Saved ROC curve to {roc_path}")

# Feature importance for tree-based models
if hasattr(best_model, 'feature_importances_'):
    print("\nTop 10 Most Important Features:")
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(feature_importance.head(10).to_string(index=False))
    
    # Plot feature importance
    plt.figure(figsize=(10, 6))
    top_features = feature_importance.head(10)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Importance')
    plt.title(f'{best_model_name} - Top 10 Feature Importance')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    feature_plot_path = 'reports/top_features_best_model.png'
    plt.savefig(feature_plot_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"✓ Saved feature importance plot to {feature_plot_path}")

# -----------------------------
# 8. Save Final Results
# -----------------------------
print("\n" + "=" * 80)
print("8. SAVING FINAL RESULTS AND ARTIFACTS")
print("=" * 80)

# Save best model
best_model_path = 'models/task5/best_model.pkl'
joblib.dump(best_model, best_model_path)
print(f"✓ Saved best model to {best_model_path}")

# Save results summary
results_path = 'reports/model_comparison_results.csv'
results_df.to_csv(results_path, index=False)
print(f"✓ Saved model comparison results to {results_path}")

# Save metrics summary
metrics_summary = {
    'best_model': best_model_name,
    'best_model_metrics': best_model_metrics,
    'training_timestamp': datetime.now().isoformat(),
    'data_summary': {
        'total_customers': len(df),
        'train_samples': len(X_train),
        'test_samples': len(X_test),
        'feature_count': X.shape[1],
        'class_distribution': y.value_counts().to_dict()
    },
    'model_performance': results_df.to_dict('records')
}

metrics_path = 'reports/metrics_summary.json'
with open(metrics_path, 'w') as f:
    json.dump(metrics_summary, f, indent=2, default=str)
print(f"✓ Saved metrics summary to {metrics_path}")

# Save prediction examples
predictions_df = pd.DataFrame({
    'customer_id': df.loc[X_test.index, 'CustomerId'].values,
    'actual_risk': y_test.values,
    'predicted_risk': y_pred_best,
    'risk_probability': y_pred_proba_best
})
predictions_path = 'reports/sample_predictions.csv'
predictions_df.head(100).to_csv(predictions_path, index=False)
print(f"✓ Saved sample predictions to {predictions_path}")

# -----------------------------
# 9. Create Unit Tests
# -----------------------------
print("\n" + "=" * 80)
print("9. CREATING UNIT TESTS")
print("=" * 80)

# Create test directory if not exists
os.makedirs('tests', exist_ok=True)

# Create a simple unit test file
test_code = '''
"""
Unit tests for Task 5 - Model Training
"""

import unittest
import pandas as pd
import numpy as np
import joblib
import os

class TestModelTraining(unittest.TestCase):
    
    def setUp(self):
        """Load test data and model"""
        self.model_path = 'models/task5/best_model.pkl'
        self.data_path = 'data/processed/task5/X_test.csv'
        self.target_path = 'data/processed/task5/y_test.csv'
    
    def test_model_exists(self):
        """Test that the best model file exists"""
        self.assertTrue(os.path.exists(self.model_path), 
                       f"Model file not found at {self.model_path}")
    
    def test_data_exists(self):
        """Test that test data exists"""
        self.assertTrue(os.path.exists(self.data_path), 
                       f"Test data not found at {self.data_path}")
        self.assertTrue(os.path.exists(self.target_path), 
                       f"Test target not found at {self.target_path}")
    
    def test_model_loading(self):
        """Test that the model can be loaded"""
        if os.path.exists(self.model_path):
            model = joblib.load(self.model_path)
            self.assertIsNotNone(model, "Failed to load model")
    
    def test_prediction_shape(self):
        """Test that predictions have correct shape"""
        if os.path.exists(self.model_path) and os.path.exists(self.data_path):
            model = joblib.load(self.model_path)
            X_test = pd.read_csv(self.data_path)
            predictions = model.predict(X_test)
            self.assertEqual(len(predictions), len(X_test), 
                           "Predictions shape doesn't match test data")
    
    def test_feature_count(self):
        """Test that feature count is consistent"""
        if os.path.exists(self.data_path):
            X_test = pd.read_csv(self.data_path)
            # Assuming model expects 10 features based on Task 4
            self.assertEqual(X_test.shape[1], 10, 
                           f"Expected 10 features, got {X_test.shape[1]}")

if __name__ == '__main__':
    unittest.main()
'''

test_file_path = 'tests/test_model_training.py'
with open(test_file_path, 'w') as f:
    f.write(test_code)

print(f"✓ Created unit tests at {test_file_path}")
print("  To run tests: python -m pytest tests/test_model_training.py")

# -----------------------------
# 10. Summary Report
# -----------------------------
print("\n" + "=" * 80)
print("TASK 5 COMPLETE - SUMMARY REPORT")
print("=" * 80)

print(f"\n✅ ALL TASK REQUIREMENTS SATISFIED:")
requirements = [
    ("Setup MLflow and pytest", "✓"),
    ("Data preparation and train-test split", "✓"),
    ("Train at least 2 models", f"✓ ({len(models)} models trained)"),
    ("Hyperparameter tuning with GridSearchCV", "✓"),
    ("Experiment tracking with MLflow", "✓"),
    ("Model evaluation with metrics", "✓"),
    ("Unit tests created", "✓"),
    ("Best model registered in MLflow", "✓")
]

for req, status in requirements:
    print(f"   {req:45} {status}")

print(f"\n📊 MODEL PERFORMANCE SUMMARY:")
print(f"   Best Model: {best_model_name}")
print(f"   ROC-AUC Score: {best_model_metrics['roc_auc']:.3f}")
print(f"   Accuracy: {best_model_metrics['accuracy']:.3f}")
print(f"   F1-Score: {best_model_metrics['f1_score']:.3f}")

print(f"\n🔧 MODELS TRAINED:")
for model_name in models.keys():
    result = results_df[results_df['model_name'] == model_name].iloc[0]
    print(f"   • {model_name:20} ROC-AUC: {result['roc_auc']:.3f}")

print(f"\n💾 ARTIFACTS CREATED:")
artifacts = [
    ('models/task5/', 'Trained models and scaler'),
    ('data/processed/task5/', 'Train/test splits'),
    ('reports/', 'Evaluation reports and visualizations'),
    ('mlruns/', 'MLflow experiment tracking'),
    ('tests/test_model_training.py', 'Unit tests')
]

for path, desc in artifacts:
    if os.path.exists(path if not path.endswith('.py') else os.path.dirname(path)):
        print(f"   ✓ {path}")
        print(f"     {desc}")

print(f"\n📈 NEXT STEPS FOR TASK 6 (DEPLOYMENT):")
print(f"   1. Create FastAPI application (src/api/main.py)")
print(f"   2. Build Pydantic models for request/response")
print(f"   3. Create Dockerfile for containerization")
print(f"   4. Set up CI/CD pipeline (.github/workflows/ci.yml)")
print(f"   5. Test API endpoints")

print(f"\n🔍 TO VIEW MLFLOW EXPERIMENTS:")
print(f"   $ mlflow ui --port 5000")
print(f"   Then open: http://localhost:5000")

print(f"\n🧪 TO RUN UNIT TESTS:")
print(f"   $ python -m pytest tests/test_model_training.py -v")

print(f"\n" + "=" * 80)
print("READY FOR TASK 6 - MODEL DEPLOYMENT")
print("=" * 80)

2025/12/15 18:36:52 INFO mlflow.tracking.fluent: Experiment with name 'credit-risk-modeling' does not exist. Creating a new experiment.


TASK 5 - MODEL TRAINING AND TRACKING
Credit Risk Model with MLflow Experimentation

1. SETTING UP MLFLOW AND CONFIGURATION
----------------------------------------
MLflow Configuration:
  Tracking URI: file:./mlruns
  Experiment: credit-risk-modeling

2. LOADING AND PREPARING DATA
----------------------------------------
Loading data from data/processed/customers_with_proxy_target_fixed.csv...
✓ Loaded 3742 customer records
Features: 10 features + CustomerId + Target

Feature matrix shape: (3742, 10)
Target distribution:
is_high_risk
1    83.5%
0    16.5%
Name: proportion, dtype: object
✓ No missing values found

3. CREATING TRAIN-TEST SPLIT
----------------------------------------
Training set: 2993 samples (80.0%)
Test set: 749 samples (20.0%)

Class distribution in training set:
is_high_risk
1    83.5%
0    16.5%
Name: proportion, dtype: object

Class distribution in test set:
is_high_risk
1    83.6%
0    16.4%
Name: proportion, dtype: object
✓ Saved train/test splits to data/proces

2025/12/15 18:41:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



  ✅ Training complete for LogisticRegression
     Best CV Score: 0.991
     Test Accuracy: 0.965
     Test ROC-AUC: 0.989
     Best parameters: {'C': 100, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'liblinear'}

TRAINING: RandomForest
  Performing GridSearchCV...
Fitting 5 folds for each of 54 candidates, totalling 270 fits


2025/12/15 18:43:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


...


  ✅ Training complete for GradientBoosting
     Best CV Score: 1.000
     Test Accuracy: 0.997
     Test ROC-AUC: 1.000
     Best parameters: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.8}

TRAINING: DecisionTree
  Performing GridSearchCV...
Fitting 5 folds for each of 36 candidates, totalling 180 fits


2025/12/15 18:44:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



  ✅ Training complete for DecisionTree
     Best CV Score: 0.999
     Test Accuracy: 0.995
     Test ROC-AUC: 0.996
     Best parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2}

6. MODEL COMPARISON AND SELECTION

Model Performance Comparison:
        model_name  accuracy  precision   recall  f1_score  roc_auc
      RandomForest  0.994660   0.998397 0.995208  0.996800 0.999818
  GradientBoosting  0.997330   0.996815 1.000000  0.998405 0.999649
      DecisionTree  0.994660   0.998397 0.995208  0.996800 0.995857
LogisticRegression  0.965287   0.996689 0.961661  0.978862 0.989117

🎯 BEST MODEL: RandomForest
   ROC-AUC: 1.000
   Accuracy: 0.995
   F1-Score: 0.997


2025/12/15 18:45:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'CreditRiskModel'.
2025/12/15 18:45:11 WARNING mlflow.tracking._model_registry.fluent: Run with id aea73f0a6acf4a73990373341327014f has no artifacts at artifact path 'best_model', registering model based on models:/m-e91d9f5efa454755a48a4a56bf51516d instead
Created version '1' of model 'CreditRiskModel'.


✓ Registered RandomForest as 'CreditRiskModel' in MLflow Model Registry

7. DETAILED EVALUATION OF BEST MODEL: RandomForest

Classification Report:
              precision    recall  f1-score   support

    Low Risk       0.98      0.99      0.98       123
   High Risk       1.00      1.00      1.00       626

    accuracy                           0.99       749
   macro avg       0.99      0.99      0.99       749
weighted avg       0.99      0.99      0.99       749


Confusion Matrix:
                Predicted
                Low  High
Actual Low     122     1
Actual High      3   623
✓ Saved ROC curve to reports/roc_curve_best_model.png

Top 10 Most Important Features:
              feature  importance
            amount_cv    0.391424
    transaction_count    0.220826
high_fraud_hour_ratio    0.145905
 customer_tenure_days    0.085075
           amount_std    0.052970
         total_amount    0.033979
           avg_amount    0.033463
    frequency_per_day    0.022743
high_risk_c